In [1]:
import cv2

In [2]:
def generate_datset():
    face_classifier= cv2.CascadeClassifier("/Users/anuragkumar/Desktop/Python/Face recognition/haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face=img[y:y+h,x:x+w]
        return cropped_face
    cap = cv2.VideoCapture(0)
    id=8
    img_id =0

    while True:
        ret,frame=cap.read()
        if face_cropped(frame) is not None:
            img_id +=1
            face = cv2.resize(face_cropped(frame),(200,200))
            face =cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            file_name_path = "Data/user." +str(id)+ "."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path,face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)

            cv2.imshow("Cropped face",face)
            if cv2.waitKey(1)==13 or int(img_id)==100:
                break
    cap.release
    cv2.destroyAllWindows()
    print("collecting samples is completed")
generate_datset()


<>:6: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
/var/folders/rg/jn52y65j5m7_v0qvtt8bc0bh0000gn/T/ipykernel_3236/343033067.py:6: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():
2024-08-14 20:12:26.493 Python[3236:190655] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


collecting samples is completed


Try and Run the ANN Model for the classifiers And train the classifiers

In [3]:
import os
import cv2
from PIL import Image  # pip install pillow
import numpy as np     # pip install numpy

def train_classifier(data_dir):
    if not os.path.exists(data_dir):
        print(f"Directory {data_dir} does not exist.")
        return
    
    # Get all file paths in the directory
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

    faces = []
    ids = []

    for image in path:
        try:
            img = Image.open(image).convert('L')  # Convert image to grayscale
            imageNp = np.array(img, 'uint8')
            
            # Extract the ID from the filename (assumes format: name.id.extension)
            id = int(os.path.split(image)[1].split(".")[1])

            faces.append(imageNp)
            ids.append(id)
        except Exception as e:
            print(f"Error processing {image}: {e}")
            continue

    if len(faces) == 0 or len(ids) == 0:
        print("No valid images found in the directory.")
        return

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    print("Training completed and classifier saved as 'classifier.xml'")

# Example usage
train_classifier("data")


Training completed and classifier saved as 'classifier.xml'


In [4]:
import cv2
import numpy as np
from PIL import Image
import os

# Function to draw boundary around detected faces and recognize them
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf, id_map):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))
        
        if confidence > 70:
            name = id_map.get(id, "UNKNOWN")
            cv2.putText(img, name, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

    return img

# Loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Initialize and load the trained model
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Mapping IDs to names
id_map = {
    1: "luv",
    2: "panda",
    3: "Shailesh",
    4: "Adarsh",
    5: "Kushal",
    6: "Himanshu jnr",
    7: "Anurag",
    8: "Vidhi"
    # Add more ID-name mappings as needed
}

# Capturing video from the default camera
video_capture = cv2.VideoCapture(0)  # Use 0 for the default webcam

while True:
    ret, img = video_capture.read()
    
    if not ret:
        print("Failed to grab frame")
        break

    img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), clf, id_map)
    cv2.imshow("Face Detection", img)
    
    if cv2.waitKey(1) == 13:  # Press Enter to exit
        break

video_capture.release()
cv2.destroyAllWindows()
